1. Visión del proyecto (para explicarlo en clase/entrevista)

“Construí un generador de datos sintéticos que usa modelos de lenguaje open-source (Llama 3.1 / 3.2 y Gemma 2) para crear datasets tabulares a partir de descripciones de negocio. El usuario define el dominio (ej. ventas, banca, salud), el tamaño del dataset y las reglas; el modelo genera datos en formato CSV. Encima de esto construí una interfaz con Gradio para que cualquier usuario pueda usarlo sin escribir código.”

Casos de uso a mencionar:

- Probar pipelines de datos cuando no hay datos reales disponibles.
- Crear datos despersonalizados para demos / prototipos.
- Augmentación de datos para ejercicios de ML.

2. Tecnologías a usar

- Modelos LLM (texto-texto):
- meta-llama/Llama-3.1-8B-Instruct 
- meta-llama/Llama-3.2-3B-Instruct 
- google/gemma-2-9b-it 

Librerías Python:

- transformers o huggingface_hub (para llamar al modelo).
- pandas (para construir el DataFrame).

gradio (UI).

- Auth Hugging Face con token. 

3. Estructura sugerida del notebook Project3-week3.ipynb

- Título + descripción del proyecto (Markdown).
- Instalación / imports.
- Configuración de modelos Hugging Face.
- Definición de esquemas de datasets (plantillas de negocio).
- Función generadora usando LLM.
- Conversión a pandas.DataFrame + validaciones básicas.
- Interfaz Gradio.

Pruebas y ejemplos de uso.

## Imports e instalación

In [ ]:
!pip install -q transformers huggingface_hub gradio pandas bitsandbytes dotenv openai torch

In [1]:
import os
import re
import pandas as pd
import torch
from huggingface_hub import login
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from dotenv import load_dotenv
from openai import OpenAI
import tempfile


/opt/conda/envs/LLM/lib/python3.11/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


## Login y carga de variables de entorno
- Carga .env.
- Toma OPENAI_API_KEY y HUGGINGFACE_API_KEY.
- Crea un cliente de OpenAI si hay API key.
- Llama a login() de Hugging Face si hay token.

In [2]:
# Load environment variables
load_dotenv(dotenv_path='/workspace/.env', override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
hf_token = os.getenv('HUGGINGFACE_API_KEY')

# Initialize OpenAI Client
if openai_api_key:
    openai_client = OpenAI(api_key=openai_api_key)
    print(" OpenAI Client Initialized")
else:
    print(" OpenAI API Key not found")

# Login to Hugging Face
if hf_token:
    if hf_token.startswith('Bearer '):
        hf_token = hf_token.replace('Bearer ', '')
    login(hf_token.strip())
    print(" Logged into Hugging Face")
else:
    print(" Hugging Face Token not found")

 OpenAI Client Initialized
 Logged into Hugging Face


## Configuración de modelos
- Definision de nombres de modelos en Hugging Face.
- En este proyecto se está usando LLAMA_3_1 en 4 bits; los demás están listos por si se usan luego.

In [3]:
# Define model identifiers
LLAMA_3_1 = "meta-llama/Llama-3.1-8B-Instruct"
LLAMA_3_2 = "meta-llama/Llama-3.2-3B-Instruct"
PHI4 = "microsoft/Phi-3-mini-4k-instruct"
GEMMA3 = "google/gemma-3-4b-it"

## Cache Management

To keep our workspace organized, we will define specific directories for each model. This prevents models from filling up the default cache partition and allows for easier management.

- Define ruta base de caché HF.
- Crea subdirectorios para cada modelo.
- Imprime rutas

In [4]:
# Base Hugging Face cache directory
hf_cache_base = os.getenv('HF_HOME', '/root/.cache/huggingface')

# Define specific cache directories for each model
model_cache_llama_3_1_8b = os.path.join(hf_cache_base, 'models', 'llama_3_1_8b')
model_cache_llama_3_2_3b = os.path.join(hf_cache_base, 'models', 'llama_3_2_3b')
model_cache_phi = os.path.join(hf_cache_base, 'models', 'phi_3_mini')
model_cache_gemma = os.path.join(hf_cache_base, 'models', 'gemma_3_4b')

# Create directories if they don't exist
os.makedirs(model_cache_llama_3_1_8b, exist_ok=True)
os.makedirs(model_cache_llama_3_2_3b, exist_ok=True)
os.makedirs(model_cache_phi, exist_ok=True)
os.makedirs(model_cache_gemma, exist_ok=True)

print(f"Llama Cache: {model_cache_llama_3_1_8b}")
print(f"Llama Cache: {model_cache_llama_3_2_3b}")
print(f"Phi-3 Cache: {model_cache_phi}")
print(f"Gemma Cache: {model_cache_gemma}")

Llama Cache: /root/.cache/huggingface/models/llama_3_1_8b
Llama Cache: /root/.cache/huggingface/models/llama_3_2_3b
Phi-3 Cache: /root/.cache/huggingface/models/phi_3_mini
Gemma Cache: /root/.cache/huggingface/models/gemma_3_4b


## Definir “plantillas de datasets”
3 dominios de negocio:
- "Retail Sales"
- "Bank Transactions"
- "Customer Support Tickets"

Cada uno tiene:

- description
- columns: lista de columnas con name, type, constraints.

In [5]:
DATASET_SCHEMAS = {
    "Retail Sales": {
        "description": "Ventas en una tienda retail de e-commerce.",
        "columns": [
            {"name": "order_id", "type": "string", "constraints": "único, formato ORD-XXXX"},
            {"name": "order_date", "type": "date", "constraints": "entre 2024-01-01 y 2024-12-31"},
            {"name": "customer_id", "type": "string", "constraints": "formato CUST-XXXX"},
            {"name": "country", "type": "category", "constraints": "Colombia, México, Chile, Perú"},
            {"name": "product_category", "type": "category", "constraints": "Electrónicos, Ropa, Hogar"},
            {"name": "unit_price", "type": "float", "constraints": "entre 5 y 200"},
            {"name": "quantity", "type": "int", "constraints": "entre 1 y 10"},
            {"name": "total_amount", "type": "float", "constraints": "unit_price * quantity"},
            {"name": "is_fraud", "type": "bool", "constraints": "True si la transacción es fraudulenta, False en caso contrario"}
        ]
    },
    "Bank Transactions": {
        "description": "Movimientos bancarios de cuentas de ahorro.",
        "columns": [
            {"name": "transaction_id", "type": "string", "constraints": "único"},
            {"name": "customer_id", "type": "string", "constraints": "formato CUST-XXXX"},
            {"name": "transaction_date", "type": "date", "constraints": "2024-01-01 a 2024-12-31"},
            {"name": "transaction_type", "type": "category", "constraints": "deposit, withdrawal, transfer"},
            {"name": "amount", "type": "float", "constraints": "entre 10 y 5_000"},
            {"name": "balance_after", "type": "float", "constraints": "saldo posterior coherente"},
            {"name": "channel", "type": "category", "constraints": "ATM, web, mobile_app, branch"}
        ]
    },
    "Customer Support Tickets": {
        "description": "Tickets de soporte para una plataforma SaaS.",
        "columns": [
            {"name": "ticket_id", "type": "string", "constraints": "único"},
            {"name": "created_at", "type": "datetime", "constraints": "2024-01-01 a 2024-12-31"},
            {"name": "customer_tier", "type": "category", "constraints": "Free, Standard, Premium"},
            {"name": "issue_type", "type": "category", "constraints": "bug, billing, onboarding, other"},
            {"name": "priority", "type": "category", "constraints": "low, medium, high, critical"},
            {"name": "resolution_time_hours", "type": "float", "constraints": ">= 0"},
            {"name": "resolved", "type": "bool", "constraints": "True/False"}
        ]
    }
}


## Construir el prompt para el LLM
### Construye el prompt en español explicándole al LLM:
- Rol (generador de datos sintéticos).
- Dataset y descripción.
- Columnas y restricciones.
- Número de filas.
- Reglas de salida: solo CSV, encabezado en primera fila, etc.

In [6]:
def build_prompt(schema_name: str, n_rows: int, extra_instructions: str = "") -> str:
    schema = DATASET_SCHEMAS[schema_name]
    lines = []

    lines.append(
        "Eres un generador de datos sintéticos tabulares para pruebas de analítica y machine learning."
    )
    lines.append(
        "Tu tarea es generar un dataset SINTÉTICO en formato CSV, sin datos personales reales."
    )
    lines.append(f"Dataset: {schema_name}")
    lines.append(f"Descripción: {schema['description']}")
    lines.append("")
    lines.append("Especificación de columnas:")

    for col in schema["columns"]:
        lines.append(
            f"- {col['name']} ({col['type']}): {col['constraints']}"
        )

    lines.append("")
    lines.append(f"Genera exactamente {n_rows} filas de datos. Es obligatorio tener {n_rows} filas.")
    lines.append("No escribas texto adicional antes ni después del CSV. Solo el CSV.")
    lines.append("Muy importante:")
    lines.append("1. La salida debe estar SOLO en formato CSV.")
    lines.append("2. La primera fila debe ser el encabezado con los nombres de las columnas.")
    lines.append("3. No incluyas explicaciones, comentarios ni texto adicional.")
    lines.append("4. Respeta tipos y rangos lo mejor posible.")
    if extra_instructions:
        lines.append("")
        lines.append("Instrucciones adicionales del usuario:")
        lines.append(extra_instructions)

    return "\n".join(lines)


## Cargar el tokenizer de Llama 3.1
- Descarga / carga el tokenizer de Llama 3.1 8B.
### Ajusta:
- pad_token = eos_token
- padding_side = "left" (útil para modelos causales).
- Aquí se define tokenizer_llama, que luego usa generate_with_local_llama.
- Si esta celda no se ejecuta antes de pulsar el botón de Gradio → NameError: tokenizer_llama is not defined.

In [7]:
tokenizer_llama = AutoTokenizer.from_pretrained(
    LLAMA_3_1,
    cache_dir=model_cache_llama_3_1_8b
)

# Ajustes recomendados para modelos causales
tokenizer_llama.pad_token = tokenizer_llama.eos_token
tokenizer_llama.padding_side = "left"

print("✅ Tokenizer loaded successfully")


✅ Tokenizer loaded successfully


## Configura  --> Quantization
### Configura BitsAndBytes para cargar el modelo:
- En 4 bits (load_in_4bit=True)
- NF4 como tipo de cuantización.
- bfloat16 como tipo de cómputo.

In [8]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

### Carga del modelo Llama 3.1 en 4-bit
- Carga el modelo en 4-bit usando quant_config, mapeando dispositivos automáticamente.
- Usa el mismo cache_dir

In [9]:
MODEL_LLAMA = AutoModelForCausalLM.from_pretrained(
    LLAMA_3_1, 
    device_map="auto", 
    quantization_config=quant_config,
    cache_dir=model_cache_llama_3_1_8b
)

print(f"✅ Model loaded successfully from: {model_cache_llama_3_1_8b}")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Model loaded successfully from: /root/.cache/huggingface/models/llama_3_1_8b


## Función generate_with_local_llama con limpieza de tensores
- Usa tokenizer_llama y MODEL_LLAMA
- Genera texto con sampling (temperature, top_p).
- Limpia memoria: del inputs, generated_ids + torch.cuda.empty_cache().

In [10]:
def generate_with_local_llama(prompt: str, max_new_tokens: int = 4096) -> str:
    # Tokenizamos el prompt
    inputs = tokenizer_llama(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True
    ).to(MODEL_LLAMA.device)

    # Generación
    with torch.no_grad():
        generated_ids = MODEL_LLAMA.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer_llama.eos_token_id
        )

    # Decodificar el texto completo (prompt + respuesta)
    full_text = tokenizer_llama.decode(
        generated_ids[0],
        skip_special_tokens=True
    )

    # Limpiar tensores intermedios para liberar memoria GPU
    del inputs, generated_ids
    torch.cuda.empty_cache()

    # Opcional: quedarte solo con lo generado después del prompt
    generated_part = full_text[len(prompt):].strip()
    return generated_part if generated_part else full_text


### Parsear la salida a DataFrame

In [11]:
import io

def parse_csv_to_df(text: str) -> pd.DataFrame:
    # Quitar posibles bloques markdown ```csv ... ```
    cleaned = re.sub(r"```(?:csv)?", "", text)
    cleaned = cleaned.strip("` \n")

    # Nos quedamos solo con las líneas que contengan comas (probable CSV)
    lines = [l for l in cleaned.splitlines() if "," in l]
    if not lines:
        print("WARN: No se encontraron líneas con comas en la salida del modelo.")
        return pd.DataFrame()

    csv_text = "\n".join(lines)

    try:
        df = pd.read_csv(io.StringIO(csv_text))
    except Exception as e:
        print("Error al parsear CSV:", e)
        print("Contenido que se intentó parsear:")
        print(csv_text[:500])
        return pd.DataFrame()

    return df


### Validaciones básicas
- ¿Número de filas = solicitado?
- ¿Hay columnas faltantes?
- ¿Tipos básicos (int, float) se pueden convertir?

In [12]:
def basic_quality_checks(df: pd.DataFrame, schema_name: str) -> dict:
    schema = DATASET_SCHEMAS[schema_name]
    expected_cols = [c["name"] for c in schema["columns"]]

    result = {
        "missing_columns": [c for c in expected_cols if c not in df.columns],
        "extra_columns": [c for c in df.columns if c not in expected_cols],
        "n_rows": len(df),
        "n_cols": df.shape[1]
    }
    return result


## Ajustar la app de datos sintéticos para usar el modelo local
Llama a:
- build_prompt
- generate_with_local_llama
- parse_csv_to_df
- basic_quality_checks

In [13]:
def synthetic_data_app(
    schema_name: str,
    n_rows: int,
    extra_instructions: str
):
    prompt = build_prompt(schema_name, n_rows, extra_instructions)
    
    # DEBUG: ver el prompt que le mandas al modelo (una vez, en consola)
    print("=== PROMPT ===")
    print(prompt[:1000])
    print("=============")

    raw_output = generate_with_local_llama(prompt)

    # DEBUG: ver lo que devuelve el modelo
    print("=== RAW OUTPUT (primeros 1000 chars) ===")
    print(raw_output[:1000])
    print("========================================")

    df = parse_csv_to_df(raw_output)
    checks = basic_quality_checks(df, schema_name)

    # DEBUG: ver tamaño del df
    print("=== DF SHAPE ===", df.shape)
    print(df.head())

    info = (
        f"Filas generadas (df): {checks['n_rows']}\n"
        f"Columnas extra: {checks['extra_columns']}\n"
        f"Columnas faltantes: {checks['missing_columns']}\n"
    )

    tmp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".csv")
    df.to_csv(tmp_file.name, index=False)
    tmp_file_path = tmp_file.name
    tmp_file.close()

    return info, df, tmp_file_path


## Interfaz Gradio (sin selector de modelo, porque usamos Llama local)
Construye la interfaz:
- Dropdown para tipo de dataset.
- Slider para número de filas.
- Textbox para instrucciones extra.
- Botón que llama synthetic_data_app.
- Muestra info, dataframe y archivo descargable.

In [14]:
with gr.Blocks(title="Synthetic Data Studio") as demo:
    gr.Markdown("# 🧪 Synthetic Data Studio\nGenerador de datos sintéticos con Llama 3.1 (4-bit)")

    schema_name = gr.Dropdown(
        choices=list(DATASET_SCHEMAS.keys()),
        value="Retail Sales",
        label="Tipo de dataset"
    )

    n_rows = gr.Slider(10, 1000, value=100, step=10, label="Número de filas")

    extra_instructions = gr.Textbox(
        lines=4,
        label="Instrucciones adicionales (opcional)",
        placeholder="Ej: Genera un 10% de transacciones fraudulentas..."
    )

    generate_btn = gr.Button("Generar datos sintéticos 🚀")

    info_out = gr.Textbox(label="Información de generación")
    df_out = gr.Dataframe(label="Vista previa del dataset")
    csv_out = gr.File(label="Descargar CSV")

    generate_btn.click(
        synthetic_data_app,
        inputs=[schema_name, n_rows, extra_instructions],
        outputs=[info_out, df_out, csv_out]
    )

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://f2916aa809dbc758cb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


=== PROMPT ===
Eres un generador de datos sintéticos tabulares para pruebas de analítica y machine learning.
Tu tarea es generar un dataset SINTÉTICO en formato CSV, sin datos personales reales.
Dataset: Bank Transactions
Descripción: Movimientos bancarios de cuentas de ahorro.

Especificación de columnas:
- transaction_id (string): único
- customer_id (string): formato CUST-XXXX
- transaction_date (date): 2024-01-01 a 2024-12-31
- transaction_type (category): deposit, withdrawal, transfer
- amount (float): entre 10 y 5_000
- balance_after (float): saldo posterior coherente
- channel (category): ATM, web, mobile_app, branch

Genera exactamente 100 filas.
Muy importante:
1. La salida debe estar SOLO en formato CSV.
2. La primera fila debe ser el encabezado con los nombres de las columnas.
3. No incluyas explicaciones, comentarios ni texto adicional.
4. Respeta tipos y rangos lo mejor posible.

Instrucciones adicionales del usuario:
Genera un 10% de transacciones fraudulentas
=== RAW OUT

Traceback (most recent call last):
  File "/opt/conda/envs/LLM/lib/python3.11/site-packages/gradio/queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/LLM/lib/python3.11/site-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/LLM/lib/python3.11/site-packages/gradio/blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/LLM/lib/python3.11/site-packages/gradio/blocks.py", line 1623, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/LLM/lib/python3.11/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend(

In [ ]:
# 🧹 Clean up GPU memory after finishing the project

import gc

# Si existieran estas variables en el espacio global, las borramos con try/except
for var_name in ["MODEL_LLAMA", "tokenizer_llama"]:
    try:
        del globals()[var_name]
        print(f"Deleted: {var_name}")
    except KeyError:
        print(f"{var_name} not found in globals()")

# Forzamos garbage collection
gc.collect()

# Limpiar cache de CUDA
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("✅ torch.cuda.empty_cache() called")
else:
    print("CUDA no está disponible en este entorno.")



In [ ]:
gr.close_all()